In [0]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

import warnings

import seaborn as sns
import matplotlib.pyplot as plt

# Deactivate all warnings
warnings.filterwarnings('ignore')

In [0]:
df = spark.sql(
    """
    select
        rt.flightkey,
        rt.route,
        rt.flight_dt,
        rt.chargeproduct,
        flt.capacity,

        -- route characteristics
        rtmap.type, 
        rtmap.region,

        -- statistics & metrics
        sum(rt.unt_net) as total_pax,
        sum(rt.rev_net) as total_rev
        
    from 
        data_experience_commercial.cbt_1423_rtsuite.master rt
    join 
        data_prod.silver_sanezdb.routemap rtmap on rt.route = rtmap.route
    join
        data_prod.silver_curated_eres.flight flt on rt.flightkey = flt.flightkey

    where 1=1 
        and rt.chargeproduct = 'Ticket'
        and rt.flight_dt between current_date() - 365 and current_date()

    group by
        rt.flightkey,
        rt.route,
        rt.flight_dt,
        rt.chargeproduct,
        flt.capacity, 
        rtmap.type, 
        rtmap.region
    """
).toPandas()

# utils

In [0]:
def waterfall_sampling(df, strata, n, min_strata_size=5, seed=None):
    """
    Perform stratified sampling using a waterfall approach.
    
    :param df: Pandas DataFrame containing the data.
    :param strata: Tuple (region, type, cap_group) defining the stratum.
    :param n: Number of samples to draw.
    :param min_strata_size: Minimum size required for a stratum, otherwise 'region' is dropped.
    :param seed: Random seed for reproducibility.
    :return: Sampled DataFrame.
    """
    region, rt_type, cap_group = strata

    # First attempt: Full stratification (type, cap_group, region)
    df_1st_level = df[(df["type"] == rt_type) & 
                      (df["cap_group"] == cap_group) & 
                      (df["region"] == region)]

    # If too small, drop 'region' and sample from (type, cap_group) level
    if len(df_1st_level) < min_strata_size:
        df_1st_level = df[(df["type"] == rt_type) & 
                          (df["cap_group"] == cap_group)]

    # Ensure we don't sample more than available
    sample_size = min(n, len(df_1st_level))

    return df_1st_level.sample(n=sample_size, random_state=seed) if sample_size > 0 else pd.DataFrame()

def repeat_waterfall_sampling(df, fixed_allocation, n_repeats=10):
    """
    Perform waterfall sampling multiple times for all fixed allocation strata.
    
    :param df: Pandas DataFrame containing the data.
    :param fixed_allocation: Dictionary with keys (region, type, cap_group) and values as sample sizes.
    :param n_repeats: Number of times to repeat the sampling.
    :return: List of sampled routes for each repeat.
    """
    samples = []

    for _ in range(n_repeats):
        seed = np.random.randint(1000)
        sampled_routes = []

        for strata, n in fixed_allocation.items():
            sampled_df = waterfall_sampling(df, strata, n, seed=seed)
            sampled_routes.extend(sampled_df.route.to_list())

        samples.append(sampled_routes)

    return samples

def compute_sample_revenue(df, samples, target_routes):
    """
    Compute the mean revenue per flight date for each sample.

    :param df: DataFrame containing 'route', 'flight_dt', and 'total_rev'.
    :param samples: List of lists, where each inner list contains sampled route names.
    :param target_routes: Tuple or list of routes to compute the target revenue.
    :return: DataFrame where each column represents a sample's mean revenue per flight date.
    """
    rev_dict = {}

    # Compute revenue for each sample
    for idx, sample in enumerate(samples):
        rev_dict[f'rev_sample{idx+1}'] = df[df['route'].isin(sample)].groupby('flight_dt')['total_rev'].mean()

    # Compute target revenue if target_routes are provided
    rev_dict['target'] = df[df['route'].isin(target_routes)].groupby('flight_dt')['total_rev'].mean()

    return pd.DataFrame(rev_dict)

def min_max_scale_df(df):
    """
    Apply Min-Max scaling to each column of the given DataFrame.

    :param df: DataFrame to scale.
    :return: Min-Max scaled DataFrame.
    """
    return (df - df.min()) / (df.max() - df.min())

def plot_revenue_comparison(data):
    plt.figure(figsize=(10, 6))

    plt.plot(data.index, data['target'], label='Target', color='black', linewidth=2, linestyle='-', zorder=3)

    for column in data.columns[:-1]:
        plt.plot(data.index, data[column], label=column, alpha=0.4)

    plt.title("Revenue Per Flight Date (Samples vs Target)", fontsize=14)
    plt.xlabel("Flight Date", fontsize=12)
    plt.ylabel("Mean Total Revenue", fontsize=12)
    plt.legend(title="Samples", bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.show()

def compute_statistics(data, target_column='target', pred_column='counterfactual'):
    """
    Compute bias and error in percentage terms without needing to descale.

    :param data: DataFrame with scaled values between 0 and 1.
    :param target_column: The column name for the target values in the DataFrame.
    :return: Bias and Error in percentage.
    """

    bias = 100 * (data[pred_column] - data[target_column]).mean()
    error = 100 * np.abs(data[pred_column] - data[target_column]).mean()
    
    return bias, error

def plot_kde_results(insample_mape, insample_bias, test_mape, test_bias):
    # Set the plot style
    sns.set(style="whitegrid")

    # Create a figure with 4 subplots (1 row, 4 columns)
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))

    # Plot each KDE on separate subplots
    sns.kdeplot(insample_mape, label="Insample MAPE", fill=True, alpha=0.6, ax=axes[0, 0])
    axes[0, 0].set_title("Insample MAPE", fontsize=14)
    axes[0, 0].set_xlabel("Value", fontsize=12)
    axes[0, 0].set_ylabel("Density", fontsize=12)

    sns.kdeplot(insample_bias, label="Insample Bias", fill=True, alpha=0.6, ax=axes[0, 1])
    axes[0, 1].set_title("Insample Bias", fontsize=14)
    axes[0, 1].set_xlabel("Value", fontsize=12)
    axes[0, 1].set_ylabel("Density", fontsize=12)

    sns.kdeplot(test_mape, label="Test MAPE", fill=True, alpha=0.6, ax=axes[1, 0])
    axes[1, 0].set_title("Test MAPE", fontsize=14)
    axes[1, 0].set_xlabel("Value", fontsize=12)
    axes[1, 0].set_ylabel("Density", fontsize=12)

    sns.kdeplot(test_bias, label="Test Bias", fill=True, alpha=0.6, ax=axes[1, 1])
    axes[1, 1].set_title("Test Bias", fontsize=14)
    axes[1, 1].set_xlabel("Value", fontsize=12)
    axes[1, 1].set_ylabel("Density", fontsize=12)

    # Adjust layout to prevent overlapping
    plt.tight_layout()
    plt.show()




## A/A tests synthetic control

Goal: find out the level of accuracy that will yield synthetic control for A/B tests when the number of route is limited (e.g. 15) in s2 settings. 
Methodology: We will run many simulations. In the first stage, we generate a test set random sampling. In a second stage we generate many control groups which have about the same amount of route through stratified sampling. Then we fit pre-test the stratified contol / test and compute the RMSE.

In [0]:
# A/A tests parameters 
df['fligth_dt'] = pd.to_datetime(df['flight_dt']) # conversion datetime 

# create statistics df for stratified sampling
df_stats = df_stats = df_test.groupby(
    ['route', 'region', 'type']
).agg(
    {
        'capacity': 'sum',
        'total_pax': 'mean',
        'total_rev': 'mean'
    }
).reset_index()

# Compute quantiles for each (region, type) group
df_capacity_quantiles = df_stats.groupby(['region', 'type']).agg(
    capacity_25th=('capacity', lambda x: x.quantile(0.25)),
    capacity_50th=('capacity', lambda x: x.quantile(0.50)),
    capacity_75th=('capacity', lambda x: x.quantile(0.75))
).reset_index()

# Merge back to the original df_stats
df_stats = df_stats.merge(df_capacity_quantiles, on=['region', 'type'], how='left')

def cap_group(row):
    if row['capacity'] <= row['capacity_50th']:
        return 'Low'
    return 'High'
df_stats['cap_group'] = df_stats.apply(cap_group, axis=1)


In [0]:
# pre loop stuff
n_sim = 1_000
test_period = 90
insample_mape = []
insaple_bias = []
test_mape = []
test_bias = []

for _ in range(n_sim):
    # get test routes and fixed allocations 
    test_routes = np.random.choice(df_stats.route.unique(), size=15, replace=False)
    fixed_allocation = df_stats.loc[
        df_stats.route.isin(test_routes)
    ].groupby(['region', 'type', 'cap_group']).size().to_dict()

    # remove markets so as not to sample from it
    df_stats_temp = df_stats.loc[
        ~df_stats.route.isin(test_routes)
    ]
    # get stratified samples 
    samples = repeat_waterfall_sampling(df_stats_temp, fixed_allocation, n_repeats=10)

    # create dataset
    data = compute_sample_revenue(df, samples, test_routes)
    data = min_max_scale_df(data)
    data = data.dropna()
    #plot_revenue_comparison(data)

    train, test = data.iloc[:test_period], data.iloc[test_period:]
    X = train.drop(columns=['target'])
    y = train['target']
    reg = LinearRegression().fit(X, y)

    train['counterfactual'] = reg.predict(train.drop(columns=['target']))
    test['counterfactual'] = reg.predict(test.drop(columns=['target']))
    biais, error = compute_statistics(train)
    insample_mape.append(error)
    insaple_bias.append(biais)
    biais, error = compute_statistics(test)
    test_mape.append(error)
    test_bias.append(biais)


In [0]:
plot_kde_results(
    insample_mape,
    insaple_bias,
    test_mape,
    test_bias,
)